In [10]:
# LIBRARIES

!pip install pandas numpy hvplot panel geopandas shapely jupyter_bokeh
import pandas as pd
import numpy as np
import hvplot.pandas
import panel as pn
import geopandas as gpd
from shapely.geometry import Point
pn.extension('hvplot')
pn.extension('tabulator')
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

**SET UP**

In [11]:
# DATA

df_cotonou_imputed = pd.read_csv('cotonou_clean.csv',encoding='iso-8859-1')
df_ottawa_imputed = pd.read_csv('ottawa_clean.csv',encoding='iso-8859-1')
df_parakou_imputed = pd.read_csv('parakou_clean.csv',encoding='iso-8859-1')
df_plaisance_imputed = pd.read_csv('plaisance_clean.csv',encoding='iso-8859-1')
df_vacoas_imputed = pd.read_csv('vacoas_clean.csv',encoding='iso-8859-1')
df_vancouver_imputed = pd.read_csv('vancouver_clean.csv', encoding='iso-8859-1')

# fichiers ville_2.csv = fichiers dont les colonnes ont été uniformisées directement dans Excel
df_cotonou_original = pd.read_csv('cotonou_2.csv')
df_ottawa_original = pd.read_csv('ottawa_2.csv')
df_parakou_original = pd.read_csv('parakou_2.csv')
df_plaisance_original = pd.read_csv('plaisance_2.csv')
df_vacoas_original = pd.read_csv('vacoas_2.csv')
df_vancouver_original = pd.read_csv('vancouver_2.csv')

# ---- FIX DATE FORMATS FOR AFRICAN CITIES (DD/MM/YYYY) ----
df_cotonou_imputed['Date']   = pd.to_datetime(df_cotonou_imputed['Date'],   format='%d/%m/%Y', errors='coerce')
df_parakou_imputed['Date']   = pd.to_datetime(df_parakou_imputed['Date'],   format='%d/%m/%Y', errors='coerce')
df_plaisance_imputed['Date'] = pd.to_datetime(df_plaisance_imputed['Date'], format='%d/%m/%Y', errors='coerce')
df_vacoas_imputed['Date']    = pd.to_datetime(df_vacoas_imputed['Date'],    format='%d/%m/%Y', errors='coerce')

df_cotonou_original['Date']   = pd.to_datetime(df_cotonou_original['Date'],   format='%d/%m/%Y', errors='coerce')
df_parakou_original['Date']   = pd.to_datetime(df_parakou_original['Date'],   format='%d/%m/%Y', errors='coerce')
df_plaisance_original['Date'] = pd.to_datetime(df_plaisance_original['Date'], format='%d/%m/%Y', errors='coerce')
df_vacoas_original['Date']    = pd.to_datetime(df_vacoas_original['Date'],    format='%d/%m/%Y', errors='coerce')

# ---- CANADIAN CITIES USE DEFAULT (MM/DD/YYYY) ----
df_ottawa_imputed['Date']    = pd.to_datetime(df_ottawa_imputed['Date'],    errors='coerce')
df_vancouver_imputed['Date'] = pd.to_datetime(df_vancouver_imputed['Date'], errors='coerce')

df_ottawa_original['Date']    = pd.to_datetime(df_ottawa_original['Date'],    errors='coerce')
df_vancouver_original['Date'] = pd.to_datetime(df_vancouver_original['Date'], errors='coerce')

# add NaN
df_ottawa_imputed['pressure'] = np.nan         # Canada has no pressure
df_vancouver_imputed['pressure'] = np.nan         # Canada has no pressure
df_vacoas_imputed['total_snow_cm'] = np.nan    # Others have no snow
df_cotonou_imputed['total_snow_cm'] = np.nan    # Others have no snow
df_parakou_imputed['total_snow_cm'] = np.nan    # Others have no snow
df_plaisance_imputed['total_snow_cm'] = np.nan    # Others have no snow

df_ottawa_original['pressure'] = np.nan         # Canada has no pressure
df_vancouver_original['pressure'] = np.nan         # Canada has no pressure
df_vacoas_original['total_snow_cm'] = np.nan    # Others have no snow
df_cotonou_original['total_snow_cm'] = np.nan    # Others have no snow
df_parakou_original['total_snow_cm'] = np.nan    # Others have no snow
df_plaisance_original['total_snow_cm'] = np.nan    # Others have no snow

In [12]:
# COMBINE

# + 'City' column to each dataset
df_cotonou_imputed['City'] = 'Cotonou'
df_parakou_imputed['City'] = 'Parakou'
df_plaisance_imputed['City'] = 'Plaisance'
df_vacoas_imputed['City'] = 'Vacoas'
df_ottawa_imputed['City'] = 'Ottawa'
df_vancouver_imputed['City'] = 'Vancouver'

df_cotonou_original['City'] = 'Cotonou'
df_parakou_original['City'] = 'Parakou'
df_plaisance_original['City'] = 'Plaisance'
df_vacoas_original['City'] = 'Vacoas'
df_ottawa_original['City'] = 'Ottawa'
df_vancouver_original['City'] = 'Vancouver'

# combination
df_all_imputed = pd.concat([df_cotonou_imputed, df_parakou_imputed, df_plaisance_imputed, df_vacoas_imputed, df_ottawa_imputed, df_vancouver_imputed],
                   ignore_index=True)

df_all_original = pd.concat([df_cotonou_original, df_parakou_original, df_plaisance_original, df_vacoas_original, df_ottawa_original, df_vancouver_original],
                   ignore_index=True)

# YearMonth col for monthly aggregation
df_all_imputed['YearMonth'] = df_all_imputed['Date'].dt.to_period('M').dt.to_timestamp()
df_all_original['YearMonth'] = df_all_original['Date'].dt.to_period('M').dt.to_timestamp()

# check
print(df_all_imputed['City'].unique())
print(df_all_original['City'].unique())

['Cotonou' 'Parakou' 'Plaisance' 'Vacoas' 'Ottawa' 'Vancouver']
['Cotonou' 'Parakou' 'Plaisance' 'Vacoas' 'Ottawa' 'Vancouver']


In [13]:
# proper datetime
df_all_imputed["Date"] = pd.to_datetime(df_all_imputed["Date"], errors="coerce")
df_all_original["Date"] = pd.to_datetime(df_all_original["Date"], errors="coerce")

# invalid dates
df_all_imputed = df_all_imputed.dropna(subset=["Date"])
df_all_original = df_all_original.dropna(subset=["Date"])

In [14]:
# colonnes
numeric_cols = ['Mean Temp (°C)', 'Min Temp (°C)', 'Max Temp (°C)',
                'Total Precip (mm)', 'Spd of Max Gust (km/h)',
                'Pressure', 'Total Snow (cm)']

# couleurs
city_colors = {
    'Ottawa': '#ff7f0e',
    'Vancouver': '#d62728',
    'Cotonou': '#2ca02c',
    'Parakou': '#32cd32',
    'Plaisance': '#1f77b4',
    'Vacoas': '#17becf'
}

**ONGLET 1**

In [20]:
# function aggregate
def aggregate_data(variable, granularity, selected_cities):
    df_filtered = df_all_imputed[df_all_imputed['City'].isin(selected_cities)].copy()
    if granularity == 'Day':
        df_filtered['Date2'] = df_filtered['Date']; freq = 'D'
    elif granularity == 'Month':
        df_filtered['Date2'] = df_filtered['Date'].dt.to_period('M').dt.to_timestamp(); freq = 'MS'
    elif granularity == 'Year':
        df_filtered['Date2'] = df_filtered['Date'].dt.to_period('Y').dt.to_timestamp(); freq = 'YS'
    all_dates = pd.date_range(df_filtered['Date2'].min(), df_filtered['Date2'].max(), freq=freq)
    grid = pd.MultiIndex.from_product([all_dates, selected_cities], names=['Date2','City']).to_frame(index=False)
    df_agg = df_filtered.groupby(['Date2','City'])[variable].mean().reset_index()
    df_full = grid.merge(df_agg, on=['Date2','City'], how='left')
    df_full = df_full.rename(columns={'Date2':'Date'})
    return df_full

# table
def stats_table(variable, granularity, selected_cities):
    df_agg = aggregate_data(variable, granularity, selected_cities)
    stats = df_agg.groupby('City')[variable].agg(['min','max','mean']).round(1).reset_index()
    return stats

# line plot
def overview_plot(variable, granularity, selected_cities):
    df_plot = aggregate_data(variable, granularity, selected_cities)
    plot_colors = [city_colors[c] for c in df_plot['City'].unique()]
    return df_plot.hvplot(
        x='Date',
        y=variable,
        by='City',
        color=plot_colors,
        width=600,
        height=400,
        legend='top',
        title=f"{variable} over Time ({granularity})"
    )

# widgets
variable_selector = pn.widgets.Select(
    options=numeric_cols, value='Mean Temp (°C)', name='', width=200
)
city_selector = pn.widgets.CheckBoxGroup(
    name='Cities',
    options=list(df_all_imputed['City'].unique()),
    value=list(df_all_imputed['City'].unique()),
    inline=True
)
granularity_selector = pn.widgets.RadioButtonGroup(
    name='Granularity',
    options=['Day','Month','Year'],
    button_type='default',
    value='Month'
)

# layout
title = pn.pane.Markdown(
    "<h1 style='text-align: center;'>APERÇU GÉNÉRAL</h1>",
    margin=(0,0,20,0)
)

# range 1: filters
filters_row = pn.Row(
    variable_selector,
    city_selector,
    granularity_selector,
    sizing_mode='stretch_width'
)

# range 2: line plot + table
plot_panel = pn.bind(overview_plot, variable_selector, granularity_selector, city_selector)

# create stats_panel as a Panel DataFrame with custom width/height
stats_panel = pn.panel(
    stats_table(
        variable_selector.value,
        granularity_selector.value,
        city_selector.value
    ),
    width=250,   # <-- set table width here
    height=350   # <-- set table height here
)

# watch for changes in widgets to update stats_panel
def update_stats(event):
    stats_panel.object = stats_table(
        variable_selector.value,
        granularity_selector.value,
        city_selector.value
    )

variable_selector.param.watch(update_stats, 'value')
granularity_selector.param.watch(update_stats, 'value')
city_selector.param.watch(update_stats, 'value')

plot_and_stats_row = pn.Row(
    pn.panel(plot_panel),
    stats_panel
)

# assemblage
dashboard1 = pn.Column(
    title,
    filters_row,
    plot_and_stats_row,
    sizing_mode='stretch_width'
)

# affichage
dashboard1_centered = pn.Row(
    pn.Column(dashboard1, width=1000),  # same as dashboard2
    sizing_mode='fixed',
    align='center'
)


dashboard1_centered.servable()

Row(align='center', sizing_mode='fixed')
    [0] Column(width=1000)
        [0] Column(sizing_mode='stretch_width')
            [0] Markdown(str, margin=(0, 0, 20, 0), sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
                [0] Select(options=['Mean Temp (°C)', ...], value='Mean Temp (°C)', width=200)
                [1] CheckBoxGroup(inline=True, name='Cities', options=['Cotonou', 'Parakou', ...], sizing_mode='stretch_width', value=['Cotonou', 'Parakou', ...])
                [2] RadioButtonGroup(name='Granularity', options=['Day', 'Month', 'Year'], sizing_mode='stretch_width', value='Month')
            [2] Row(sizing_mode='stretch_width')
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
                [1] DataFrame(DataFrame, height=350, sizing_mode='fixed', width=250)

**ONGLET 2**

In [22]:
# widgets

# define CSS
filter_css = ['filters-big']

pn.extension(raw_css=[
    """
    .filters-big .bk-widget-label {
        font-size: 18px;
        font-weight: bold;
    }

    .filters-big select,
    .filters-big .bk-checkbox {
        font-size: 16px;
    }

    /* Do NOT target generic `.bk-` classes inside DateRangeSlider */
    .date-slider-big .bk-input {
        font-size: 16px;
    }
    """
])

# widgets
variable_selector_eda = pn.widgets.Select(
    options=numeric_cols,
    value='Mean Temp (°C)',
    width=200  # adjust width as needed
)

city_selector_eda = pn.widgets.RadioButtonGroup(
    name='City',
    options=list(df_all_imputed['City'].unique()),
    value='Ottawa',  # default
    button_type='default'
)

date_slider_eda = pn.widgets.DateRangeSlider(
    name='Date Range',
    start=df_all_imputed['Date'].min(),
    end=df_all_imputed['Date'].max(),
    value=(df_all_imputed['Date'].min(), df_all_imputed['Date'].max()),
    width=250
)

chart_type_selector = pn.widgets.RadioButtonGroup(
    name='Chart Type',
    options=['Line', 'Histogram', 'Barplot'],
    button_type='default',
    value='Line'
)

# 3) Apply CSS classes AFTER widget creation
variable_selector_eda.css_classes = filter_css
city_selector_eda.css_classes = filter_css
date_slider_eda.css_classes = ['date-slider-big']
chart_type_selector.css_classes = filter_css

# data filtering

def filter_data(city, var, date_range):
    start, end = date_range
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)

    # Wrap city in list to keep old logic working
    df = df_all_imputed[
        (df_all_imputed['City'].isin([city])) &
        (df_all_imputed['Date'] >= start) &
        (df_all_imputed['Date'] <= end)
    ]
    return df

# time series
def eda_timeseries(city, var, date_range):
    df = filter_data(city, var, date_range)
    colors = [city_colors[city]]  # single city
    return df.hvplot(
        x='Date', y=var, by='City',
        color=colors,
        height=350,
        legend='top',
        line_width=2,
        title=f"{var} over Time",
        sizing_mode='stretch_width'  # <- make width flexible
    )

# histogram
def eda_histogram(city, var, date_range):
    df = filter_data(city, var, date_range)
    color = city_colors[city]
    return df.hvplot.hist(
        y=var,
        bins=20,
        alpha=0.5,
        color=color,
        legend_label=city,
        height=300,
        title=f"{var} Histogram — {city}",
        sizing_mode='stretch_width'
    )

# bar plot
def eda_barplot(city, var, date_range):
    df = filter_data(city, var, date_range)
    df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
    df_agg = df.groupby(['City','Month'])[var].mean().reset_index()
    colors = [city_colors[city]]  # instead of iterating over city
    return df_agg.hvplot.bar(
        x='Month', y=var, by='City',
        color=colors,
        height=350,
        legend='top',
        title=f"{var} Monthly Trend",
        sizing_mode='stretch_width'  # responsive width
    )

# heatmap

# --- Correlation Heatmap Helper ---
def compute_corr_heatmap(df):
    corr = df.corr(numeric_only=True)

    corr_long = (
        corr.reset_index()
            .melt(id_vars='index', var_name='columns', value_name='value')
            .rename(columns={'index': 'rows'})
    )

    return corr_long.hvplot.heatmap(
        x='',
        y='',
        C='value',
        cmap='RdBu_r',
        width=400,
        height=350,
        title='Correlation Heatmap'
    ).opts(invert_yaxis=True)


# --- EDA correlation function ---
def eda_correlation(cities, variable, date_range):
    start, end = date_range
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)

    df_city = df_all_imputed[
        (df_all_imputed['City'].isin([cities])) &
        (df_all_imputed['Date'] >= start) &
        (df_all_imputed['Date'] <= end)
    ].copy()

    num_cols = df_city[numeric_cols].dropna(axis=1, how='all')
    if num_cols.empty:
        return pn.pane.Markdown("### No numeric data available", style={'color': 'red'})

    corr = num_cols.corr()

    # Keep the column names exactly as in the dataframe
    corr_long = corr.reset_index().melt(id_vars='index', value_name='value')
    corr_long = corr_long.rename(columns={'index': 'rows', 'variable': 'columns'})

    return corr_long.hvplot.heatmap(
        x='columns',
        y='rows',
        C='value',
        cmap='RdBu_r',
        width=400,
        height=350,
        title=f"Correlation Heatmap — {cities}"
    ).opts(
        invert_yaxis=True,
        xlabel='',  # remove x-axis label
        ylabel='',  # remove y-axis label
        xrotation=90  # rotate x-axis labels
    )

# chart selection
def eda_chart(city, var, date_range, chart_type):
    start, end = date_range
    start = pd.Timestamp(start)
    end = pd.Timestamp(end)

    if chart_type == 'Line':
        return eda_timeseries(city, var, (start, end))
    elif chart_type == 'Histogram':
        return eda_histogram(city, var, (start, end))
    elif chart_type == 'Barplot':
        return eda_barplot(city, var, (start, end))


# layout
# Top row: all filters in one line
filters_row = pn.Row(
    variable_selector_eda,
    date_slider_eda,
    city_selector_eda,
    chart_type_selector,
    sizing_mode='stretch_width',
    margin=(10,5)  # optional spacing around filters
)

# Bottom row: Heatmap | Chart
bottom_row = pn.Row(
    pn.bind(eda_correlation, city_selector_eda, variable_selector_eda, date_slider_eda),
    pn.bind(eda_chart, city_selector_eda, variable_selector_eda, date_slider_eda, chart_type_selector),
    sizing_mode='stretch_width'
)

# Main layout
dashboard2 = pn.Column(
    pn.pane.HTML("<h1 style='text-align:center'>ANALYSE PAR VILLE</h1>", sizing_mode='stretch_width'),
    filters_row,
    bottom_row,
    sizing_mode='stretch_width'
)

# affichage
dashboard2_centered = pn.Row(
    pn.Column(dashboard2, width=1400),  # fix max width
    sizing_mode='stretch_width',             # row stretches to full width
    align='center'                           # center the column in the row
)

dashboard2_centered.servable()

Row(align='center', sizing_mode='stretch_width')
    [0] Column(width=1400)
        [0] Column(sizing_mode='stretch_width')
            [0] HTML(str, sizing_mode='stretch_width')
            [1] Row(margin=(10, 5), sizing_mode='stretch_width')
                [0] Select(css_classes=['filters-big'], options=['Mean Temp (°C)', ...], value='Mean Temp (°C)', width=200)
                [1] DateRangeSlider(css_classes=['date-slider-big'], end=Timestamp('2024-12-31 0..., name='Date Range', start=Timestamp('2020-01-01 0..., value=(Timestamp('2020-01-01 00:..., value_end=Timestamp('2024-12-31 0..., value_start=Timestamp('2020-01-01 0..., width=250)
                [2] RadioButtonGroup(css_classes=['filters-big'], name='City', options=['Cotonou', 'Parakou', ...], sizing_mode='stretch_width', value='Ottawa')
                [3] RadioButtonGroup(css_classes=['filters-big'], name='Chart Type', options=['Line', 'Histogram', ...], sizing_mode='stretch_width', value='Line')
            [2] Row(sizing_mode='stretch_width')
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
                [1] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')

**ONGLET 3**

In [17]:
# --- Widgets ---
cities = sorted(df_all_original['City'].unique())
city_selector = pn.widgets.Select(name='', options=cities, value=cities[0])

variables = ['Mean Temp (°C)', 'Min Temp (°C)', 'Max Temp (°C)',
             'Total Precip (mm)', 'Spd of Max Gust (km/h)',
             'Pressure', 'Total Snow (cm)']
variable_selector = pn.widgets.Select(name='', options=variables, value=variables[0])

# --- Function to create the plot ---
def plot_avant_apres(city, variable):
    df_ori = df_all_original[df_all_original['City'] == city]
    df_imp = df_all_imputed[df_all_imputed['City'] == city]

    p1 = df_ori.hvplot.line(x='Date', y=variable, color='red', label='Avant imputation')
    p2 = df_imp.hvplot.line(x='Date', y=variable, color='green', label='Après imputation')

    return (p1 * p2).opts(
        title=f"{variable} – {city}",
        xlabel="Date",
        ylabel=variable,
        legend_position='top_left',
        width=700,
        height=400
    )

# --- Vertical stats table with title ---
def stats_table_missing_vertical(city, variable):
    df_ori = df_all_original[df_all_original['City'] == city]
    df_imp = df_all_imputed[df_all_imputed['City'] == city]

    stats_df = pd.DataFrame({
        '': [
            df_ori[variable].isna().sum(),
            df_imp[variable].isna().sum()
        ]
    }, index=["Avant imputation", "Après imputation"])

    return pn.Column(
        pn.pane.Markdown("### <center>Valeurs manquantes</center>"),
        pn.widgets.Tabulator(
            stats_df,
            show_index=True,
            width=200,
            height=80
        )
    )

# --- Assemble dashboard ---
dashboard3 = pn.Column(
    pn.pane.HTML("<h2 style='text-align:center'>AVANT / APRÈS IMPUTATION</h2>"),
    pn.Row(city_selector, variable_selector),
    pn.Row(
        pn.panel(
            pn.bind(plot_avant_apres, city=city_selector, variable=variable_selector),
            width=700
        ),
        pn.panel(
            pn.bind(stats_table_missing_vertical, city=city_selector, variable=variable_selector),
            width=200,
            margin=(0, 0, 0, 40)  # push table slightly to the right
        )
    )
)

# Center the dashboard
dashboard3_centered = pn.Row(
    pn.Column(dashboard3, width=1000),
    sizing_mode='stretch_width',
    align='center'
)

dashboard3_centered.servable()

Row(align='center', sizing_mode='stretch_width')
    [0] Column(width=1000)
        [0] Column(sizing_mode='stretch_width')
            [0] HTML(str, sizing_mode='stretch_width')
            [1] Row(sizing_mode='stretch_width')
                [0] Select(options=['Cotonou', 'Ottawa', ...], sizing_mode='stretch_width', value='Cotonou')
                [1] Select(options=['Mean Temp (°C)', ...], sizing_mode='stretch_width', value='Mean Temp (°C)')
            [2] Row(sizing_mode='stretch_width')
                [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)
                [1] ParamFunction(function, _pane=Column, defer_load=False, margin=(0, 0, 0, 40), width=200)

**DASHBOARD**

In [21]:
import panel as pn

pn.extension(sizing_mode="stretch_width")

# CREATE THE TABS
onglets = pn.Tabs(
    ("ONGLET 1 - APERÇU GÉNÉRAL", dashboard1_centered),
    ("ONGLET 2 - ANALYSE PAR VILLE", dashboard2_centered),
    ("ONGLET 3 - AVANT / APRÈS IMPUTATION", dashboard3_centered)
)

# SERVE THE TABS
onglets.servable()

numpy.exceptions.DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes._PyFloatDType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes._PyFloatDType'>)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
 File "/usr/local/lib/python3.12/dist-packages/pyviz_comms/__init__.py", line 341, in _handle_msg
 self._on_msg(msg)
 File "/usr/local/lib/python3.12/dist-packages/panel/viewable.py", line 500, in _on_msg
 doc.unhold()
 File "/usr/local/lib/python3.12/dist-packages/bokeh/document/document.py", line 776, in unhold
 self.callbacks.unhold()
 File "/usr/local/lib/python3.12/dist-packages/bokeh/document/callbacks.py", line 441, in unhold
 self.trigger_on_change(event)
 File "/usr/local/lib/python3.12/dist-packages/bokeh/document/callbacks.py", line 418, in trigger_on_change
 invoke_with_curdoc(doc, event.callback_invoker)
 File "/usr/local/lib/python3.12/dist-packages/bokeh/document/callbacks.py", line 453, in invoke_with_curdoc
 return f()
 ^^^
 File "/usr/local/lib/python3.12/dist-packages/bokeh/util/callback_manager.py", line 174, in invoke
 callback(attr, old, new)
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 565, in _comm_change
 state._handle_exception(e)
 File "/usr/local/lib/python3.12/dist-packages/panel/io/state.py", line 496, in _handle_exception
 raise exception
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 563, in _comm_change
 self._schedule_change(doc, comm)
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 545, in _schedule_change
 self._change_event(doc)
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 541, in _change_event
 self._process_events(events)
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 495, in _process_events
 raise e
 File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
 self.param.update(**self_params)
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
 restore = dict(self_._update(arg, **kwargs))
 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2820, in _update
 self_._batch_call_watchers()
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 3037, in _batch_call_watchers
 self_._execute_watcher(watcher, events)
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2999, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.12/dist-packages/panel/param.py", line 945, in _replace_pane
 self._update_inner(new_object)
 File "/usr/local/lib/python3.12/dist-packages/panel/pane/base.py", line 751, in _update_inner
 new_pane, internal = self._update_from_object(
 ^^^^^^^^^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.12/dist-packages/panel/pane/base.py", line 740, in _update_from_object
 old_object.object = object
 ^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 628, in _f
 instance_param.__set__(obj, val)
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 630, in _f
 return f(self, obj, val)
 ^^^^^^^^^^^^^^^^^
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 1878, in __set__
 obj.param._call_watcher(watcher, event)
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 3017, in _call_watcher
 self_._execute_watcher(watcher, (event,))
 File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2999, in _execute_watcher
 watcher.fn(*args, **kwargs)
 File "/usr/local/lib/python3.12/dist-packages/panel/pane/base.py", line 431, in _update_pane


Tabs(sizing_mode='stretch_width')
    [0] Row(align='center', sizing_mode='fixed')
        [0] Column(width=1000)
            [0] Column(sizing_mode='stretch_width')
                [0] Markdown(str, margin=(0, 0, 20, 0), sizing_mode='stretch_width')
                [1] Row(sizing_mode='stretch_width')
                    [0] Select(options=['Mean Temp (°C)', ...], value='Mean Temp (°C)', width=200)
                    [1] CheckBoxGroup(inline=True, name='Cities', options=['Cotonou', 'Parakou', ...], sizing_mode='stretch_width', value=['Cotonou', 'Parakou', ...])
                    [2] RadioButtonGroup(name='Granularity', options=['Day', 'Month', 'Year'], sizing_mode='stretch_width', value='Month')
                [2] Row(sizing_mode='stretch_width')
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
                    [1] DataFrame(DataFrame, height=350, sizing_mode='fixed', width=250)
    [1] Row(align='center', sizing_mode='stretch_width')
        [0] Column(width=1400)
            [0] Column(sizing_mode='stretch_width')
                [0] HTML(str, sizing_mode='stretch_width')
                [1] Row(margin=(10, 5), sizing_mode='stretch_width')
                    [0] Select(css_classes=['filters-big'], options=['Mean Temp (°C)', ...], value='Mean Temp (°C)', width=200)
                    [1] DateRangeSlider(css_classes=['date-slider-big'], end=Timestamp('2024-12-31 0..., name='Date Range', start=Timestamp('2020-01-01 0..., value=(datetime.date(2020, ..., value_end=datetime.date(2024, ..., value_start=datetime.date(2020, 1, 1), width=250)
                    [2] RadioButtonGroup(css_classes=['filters-big'], name='City', options=['Cotonou', 'Parakou', ...], sizing_mode='stretch_width', value='Vancouver')
                    [3] RadioButtonGroup(css_classes=['filters-big'], name='Chart Type', options=['Line', 'Histogram', ...], sizing_mode='stretch_width', value='Histogram')
                [2] Row(sizing_mode='stretch_width')
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
                    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
    [2] Row(align='center', sizing_mode='stretch_width')
        [0] Column(width=1000)
            [0] Column(sizing_mode='stretch_width')
                [0] HTML(str, sizing_mode='stretch_width')
                [1] Row(sizing_mode='stretch_width')
                    [0] Select(options=['Cotonou', 'Ottawa', ...], sizing_mode='stretch_width', value='Cotonou')
                    [1] Select(options=['Mean Temp (°C)', ...], sizing_mode='stretch_width', value='Min Temp (°C)')
                [2] Row(sizing_mode='stretch_width')
                    [0] ParamFunction(function, _pane=HoloViews, defer_load=False, width=700)
                    [1] ParamFunction(function, _pane=Column, defer_load=False, margin=(0, 0, 0, 40), width=200)

2025-11-27 20:05:05,644 ERROR: panel.reactive - Callback failed for object named 'City' changing property {'value': 'Plaisance'} 
numpy.exceptions.DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes._PyFloatDType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes._PyFloatDType'>)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-

2025-11-27 20:05:12,741 ERROR: panel.reactive - Callback failed for object named 'City' changing property {'value': 'Vancouver'} 
numpy.exceptions.DTypePromotionError: The DType <class 'numpy.dtypes.DateTime64DType'> could not be promoted by <class 'numpy.dtypes._PyFloatDType'>. This means that no common DType exists for the given inputs. For example they cannot be stored in a single array unless the dtype is `object`. The full list of DTypes is: (<class 'numpy.dtypes.DateTime64DType'>, <class 'numpy.dtypes._PyFloatDType'>)

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-

/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()


...

2025-11-27 20:15:59,019 ERROR: panel.reactive - Callback failed for object named 'Date Range' changing property {'value': (datetime.date(2021, 12, 28), datetime.date(2021, 12, 28))} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2820, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 3037, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2999, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/usr/local/lib/python3.12/dist-pa

2025-11-27 20:16:17,023 ERROR: panel.reactive - Callback failed for object named 'Cities' changing property {'value': []} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2820, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 3037, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2999, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/tmp/ipython-input-2416141368.py", line 85, in update_stats
    stats_panel.object = stats_ta

2025-11-27 20:16:31,781 ERROR: panel.reactive - Callback failed for object named 'Cities' changing property {'value': []} 
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/panel/reactive.py", line 476, in _process_events
    self.param.update(**self_params)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2787, in update
    restore = dict(self_._update(arg, **kwargs))
                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2820, in _update
    self_._batch_call_watchers()
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 3037, in _batch_call_watchers
    self_._execute_watcher(watcher, events)
  File "/usr/local/lib/python3.12/dist-packages/param/parameterized.py", line 2999, in _execute_watcher
    watcher.fn(*args, **kwargs)
  File "/tmp/ipython-input-2416141368.py", line 85, in update_stats
    stats_panel.object = stats_ta

/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
/tmp/ipython-input-3553231451.py:107: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Month'] = df['Date'].dt.to_period('M').dt.to_timestamp()
